In [7]:
from glob import glob
from explicit_memory.utils import read_yaml, read_pickle
import pandas as pd

table = {1: [], 6: [], 12: [], 24: [], 48: [], 50: [], 100: []}
for results_path in sorted(
    glob("./training_results/baselines/dqn_lstm/*/*/results.yaml")
):
    train_path = results_path.replace("results.yaml", "train.yaml")
    handcrafted_path = results_path.replace("results.yaml", "handcrafted.yaml")

    results = read_yaml(results_path)
    train = read_yaml(train_path)
    handcrafted = read_yaml(handcrafted_path)

    test_score = results["test_score"]
    sorted_val = sorted(results["validation_score"], key=lambda x: x["mean"])
    val_score = sorted_val[-1]

    try:
        table[train["history_block_size"]].append(
            {
                "test_mean": test_score["mean"],
                "test_std": test_score["std"],
                "val_mean": val_score["mean"],
                "val_std": val_score["std"],
                # "gamma": train["gamma"],
                # "pretrain_semantic": train["pretrain_semantic"],
                "batch_size": train["batch_size"],
                "dir": results_path.split("/")[-2],
                "test_seed": train["test_seed"],
                # "target_update_interval": train["target_update_interval"],
                # "mm_agent_path": train["mm_agent_path"].split("/")[-2],
                # "question_interval": train["env_config"]["question_interval"],
                # "num_iterations": train["num_iterations"],
                # "replay_buffer_size": train["replay_buffer_size"],
                # "warm_start": train["warm_start"],
                # "batch_size": train["batch_size"],
                "handcrafted_random": handcrafted["random"]["mean"],
                "handcrafted_random_std": handcrafted["random"]["std"],
                "handcrafted_avoid_walls": handcrafted["avoid_walls"]["mean"],
                "handcrafted_avoid_walls_std": handcrafted["avoid_walls"]["std"],
            }
        )
    except Exception as e:
        print(e)
        pass

table[train["history_block_size"]] = sorted(
    table[train["history_block_size"]], key=lambda x: x["test_mean"], reverse=True
)
table = pd.DataFrame(table[100])
table

,test_mean,test_std,val_mean,val_std,batch_size,dir,test_seed,handcrafted_random,handcrafted_random_std,handcrafted_avoid_walls,handcrafted_avoid_walls_std
0,275.4,58.85,308.5,41.12,32,2024-01-13 13:41:44.888421,0,417.6,143.321457,498.6,61.25553
1,221.3,35.57,301.1,47.12,32,2024-01-13 13:41:44.899181,1,417.6,143.321457,498.6,61.25553
2,445.0,61.22,460.0,40.97,32,2024-01-13 23:53:11.915503,3,417.6,143.321457,498.6,61.25553
3,0.0,0.00,16.4,32.63,32,2024-01-13 23:53:11.925247,2,417.6,143.321457,498.6,61.25553
4,289.6,30.93,312.2,47.63,32,2024-01-14 10:07:11.274415,4,417.6,143.321457,498.6,61.25553


In [8]:
table.val_mean.mean(), table.val_mean.std(), table.test_mean.mean(), table.test_mean.std()

(279.64, 161.3835586421368, 246.26000000000005, 160.85091855503964)

In [ ]:
from glob import glob
from explicit_memory.utils import read_yaml
import pandas as pd

table = {"l": {8: [], 16: []}, "l2": {8: [], 16: []}}
for results_path in sorted(glob("./training_results/mm/LSTM/*/results.yaml")):
    train_path = results_path.replace("results.yaml", "train.yaml")
    handcrafted_path = results_path.replace("results.yaml", "handcrafted.yaml")

    results = read_yaml(results_path)
    train = read_yaml(train_path)
    handcrafted = read_yaml(handcrafted_path)
    print(handcrafted)

    test_score = results["test_score"]
    sorted_val = sorted(results["validation_score"], key=lambda x: x["mean"])
    val_score = sorted_val[-1]

    try:
        table[train["env_config"]["room_size"]][train["capacity"]["episodic"]].append(
            {
                "test_mean": test_score["mean"],
                "test_std": test_score["std"],
                "val_mean": val_score["mean"],
                "val_std": val_score["std"],
                "gamma": train["gamma"],
                # "pretrain_semantic": train["pretrain_semantic"],
                "batch_size": train["batch_size"],
                "dir": results_path.split("/")[-2],
                "test_seed": train["test_seed"],
                # "target_update_interval": train["target_update_interval"],
                # "mm_agent_path": train["mm_agent_path"].split("/")[-2],
                "question_interval": train["env_config"]["question_interval"],
                "num_iterations": train["num_iterations"],
                "replay_buffer_size": train["replay_buffer_size"],
                "warm_start": train["warm_start"],
                # "batch_size": train["batch_size"],
            }
        )
    except Exception as e:
        print(e)
        pass

for room_size in ["l", "l2"]:
    for capacity in [8, 16]:
        table[room_size][capacity] = sorted(
            table[room_size][capacity], key=lambda x: x["test_mean"], reverse=True
        )
table = pd.DataFrame([foo for foo in table["l"][16]])
table

In [ ]:
from explicit_memory.visualize_results import save_figs_and_memories

save_figs_and_memories(
    agent_dir="./trained-agents/lstm-explore/2024-01-06 20:04:03.511403/",
    mm_policy="neural",
    qa_policy="episodic_semantic",
    explore_policy="neural",
    test_seed=1,
)

In [ ]:
from glob import glob
from explicit_memory.utils import read_yaml
import pandas as pd

table = {"l": {8: [], 16: []}, "l2": {8: [], 16: []}}
for results_path in sorted(glob("./training_results/mm/LSTM/*/results.yaml")):
    train_path = results_path.replace("results.yaml", "train.yaml")

    results = read_yaml(results_path)
    train = read_yaml(train_path)

    test_score = results["test_score"]
    sorted_val = sorted(results["validation_score"], key=lambda x: x["mean"])
    val_score = sorted_val[-1]

    try:
        table[train["env_config"]["room_size"]][train["capacity"]["episodic"]].append(
            {
                "test_mean": test_score["mean"],
                "test_std": test_score["std"],
                "val_mean": val_score["mean"],
                "val_std": val_score["std"],
                "gamma": train["gamma"],
                # "pretrain_semantic": train["pretrain_semantic"],
                "batch_size": train["batch_size"],
                "dir": results_path.split("/")[-2],
                "test_seed": train["test_seed"],
            }
        )
    except KeyError:
        pass

for room_size in ["l", "l2"]:
    for capacity in [8, 16]:
        table[room_size][capacity] = sorted(
            table[room_size][capacity], key=lambda x: x["test_mean"], reverse=True
        )

In [ ]:
pd.DataFrame([foo for foo in table["l"][16]])

In [ ]:
pd.DataFrame([foo for foo in table["l"][16]]).test_mean.mean(
), pd.DataFrame([foo for foo in table["l"][16]]).test_mean.std()

In [ ]:
pd.DataFrame([foo for foo in table["l"][16]]).val_mean.mean(
), pd.DataFrame([foo for foo in table["l"][16]]).val_mean.std()